## Проект: научить модель находить фейковые новости

In [ ]:
import numpy as np
import pandas as pd
import pymorphy2
from tqdm import tqdm
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from pymystem3 import Mystem
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
sw_ru = set(stopwords.words('russian'))

In [621]:
df = pd.read_csv("train.tsv", sep="\t")
df.head()

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1


#### Предобработка данных: Лемматизация, удаление всех стоп слов, изменение заглавных букв на прописные

In [622]:
lemmatizer = pymorphy2.MorphAnalyzer()

def preprocess(sent):
    clean_sent= ' '.join([word for word in sent.split() if not word in sw_ru])
    clean_sent = clean_sent.lower()
    tokenized_sent = sent.split()
    
    return ' '.join([lemmatizer.parse(word)[0].normal_form
                    for word in tokenized_sent])

df.title = df.title.apply(preprocess)

In [623]:
df.head()

,title,is_fake
0,москвич владимир клутина прийти счёт за вмешат...,1
1,агент кокорин назвать езда по встречка житейск...,0
2,госдума рассмотреть возможность введение секре...,1
3,фас заблокировать поставка скоростной трамвай ...,0
4,против навальный завести дело о недоносительст...,1


#### Используем TF-IDF чтобы векторизовать наш текст

In [624]:
tf = TfidfVectorizer()
vectorized = tf.fit_transform(df.title)
vectorized.shape

(5758, 11700)

#### Так как после приенения tf-idf размерность обучающей выборки стала равной (5758, 11700), придется использовать Word2vec. Размерность (5758, 11700) не подходит, так как неизвестных стала больше чем самих уравнений, если рассматривать данную размерность, как размерность ЛСУ.
#### Данные содержащиеся в файле model.txt содрежат большое количество слов, где к каждому слову сопоставлен вектор длины 300. В предложении мы просто суммируем по координатно все слова. Таким образом размерность обучающий выборки будет равен (5758, 300) 

In [625]:
word2vec = {}

with open('185/model.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        word, *vec = line.split()
        word2vec[word] = np.array(vec, dtype=np.float32)

#### Чтобы взять длину вектора слова из word2vec надо узнать к какому части речи принадлежит слово

In [626]:
m = Mystem()

In [627]:
mapping = {'A': 'ADJ',
 'ADV': 'ADV',
 'ADVPRO': 'ADV',
 'ANUM': 'ADJ',
 'APRO': 'DET',
 'COM': 'ADJ',
 'CONJ': 'SCONJ',
 'INTJ': 'INTJ',
 'NONLEX': 'X',
 'NUM': 'NUM',
 'PART': 'PART',
 'PR': 'ADP',
 'S': 'NOUN',
 'SPRO': 'PRON',
 'UNKN': 'X',
 'V': 'VERB'}

def tag(word='пожар'):
    processed = m.analyze(word)[0]
    lemma = processed["analysis"][0]["lex"].lower().strip()
    pos = processed["analysis"][0]["gr"].split(',')[0]
    pos = pos.split('=')[0].strip()
    tagged = lemma+'_'+mapping[pos]
    return tagged

tag('от')

'от_ADP'

#### Проверим работу функции tag()

In [628]:
for word in df.head().title[0].split():
    print(tag(word))

москвич_NOUN
владимир_NOUN
клутина_NOUN
приходить_VERB
счет_NOUN
за_ADP
вмешательство_NOUN
в_NOUN
американский_ADJ
выборы_NOUN


In [629]:
texts = df.title.tolist()

#### К тому же, надо убрать все пунктуации, которые содержатся в словах

In [630]:
punct = '«*-#`_—:%,”.\u200e–/!»(&°“?)+$'

In [631]:
line = 0
for line in range(len(texts)):
    for i in punct:
        if i in texts[line]:   
            texts[line] = texts[line].replace(i, "")


#### Векторизуем все слова и предложения

In [632]:
vecs = []
errors = []
for line in tqdm(texts):
    vec = 0
    for word in line.split():
        try:
            word_tagged = tag(word)
        except:
            continue
        if word_tagged in word2vec:
            vec += word2vec[word_tagged]
    vecs.append(vec)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 5758/5758 [00:06<00:00, 939.40it/s]


In [633]:
# Посмотрим длину векторов
len(vecs), df.is_fake.shape

(5758, (5758,))

In [634]:
y = df.is_fake
y

0       1
1       0
2       1
3       0
4       1
       ..
5753    0
5754    0
5755    1
5756    0
5757    0
Name: is_fake, Length: 5758, dtype: int64

In [638]:
X = pd.DataFrame(np.stack(vecs.tolist()))

In [639]:
X

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.510404,-0.371637,-0.087650,-1.181582,0.808300,0.284771,-0.546974,0.237337,-0.614967,0.378761,...,0.953727,-0.320389,-0.297604,0.676478,-0.576937,0.046073,-0.779320,-0.267583,0.870950,-0.172394
1,0.403477,-0.910513,-0.976094,-1.003196,1.074816,0.936543,-0.005817,0.518956,1.071469,0.123120,...,-0.256613,-0.206250,-0.834772,0.508822,0.183094,-0.613571,-0.446144,0.033827,1.316615,-0.657072
2,0.168775,-0.061932,-1.445975,-1.939399,-0.301081,0.893585,-0.757300,-1.390678,-0.787404,1.206545,...,-1.028487,0.860131,-1.304725,1.274230,0.332891,0.856670,0.088465,1.988401,1.679559,-1.273024
3,0.049231,-1.329170,-1.064765,-2.113189,-0.241770,0.913811,0.301923,-0.744030,-1.425886,-0.000816,...,0.472606,0.826701,0.317132,-0.217095,-0.431721,-1.119653,-0.296391,1.227143,1.080529,-0.572188
4,-0.368783,-0.315182,-0.995509,-0.520499,-0.288078,0.916106,-0.650810,0.058484,1.074858,-0.007309,...,-0.477856,0.460579,-0.457197,-0.286810,0.722710,-0.422584,-0.558892,-0.581496,0.724672,-1.198699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5752,0.171561,-0.466605,0.354408,-0.434025,0.297871,-0.368645,-0.686873,0.402240,0.667732,-0.109470,...,-0.056211,-0.090652,0.910740,0.575158,-0.423393,0.227548,-1.188019,-0.630270,0.634020,-0.302923
5753,0.712983,0.747248,-0.466906,-1.627455,0.259094,0.522675,0.188684,-0.553735,-0.415958,-0.481739,...,0.746715,-0.184465,0.153195,0.594438,-0.173378,-0.654142,0.411020,0.404238,0.569542,-0.053117
5754,-2.270884,-0.715431,-0.434485,-2.560010,1.205226,1.337797,-1.840779,0.193274,0.444074,1.228262,...,1.738383,-0.699253,-1.025795,0.478820,-0.103984,1.388157,-0.396898,-0.211119,1.518825,-0.156186
5755,-0.254447,-0.294217,-0.444222,-0.605973,0.342095,0.492321,-0.107241,-1.476999,-0.134240,0.130027,...,0.384965,0.235537,-0.195102,1.464609,-0.087126,-0.686288,-0.254199,0.468867,1.209841,-0.114545


#### Делим выборку на train и test. Пусть тестовые данные содержат 20% исходной обучающий выборки

In [640]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

## Model training

### Логистическая регрессия

In [641]:
lr = LogisticRegressionCV(cv = 3)

In [642]:
lr.fit(X_train, y_train)

LogisticRegressionCV(cv=3)

In [643]:
y_pred = lr.predict(X_test)

In [644]:
f1_score(y_pred, y_test)

0.8517520215633423

#### Если дать тестовым данным 5 процентов, то метрика f1_score равна 88.6 процетнам. Если же тествым дать 10 процентов, то метрика f1_score равна 85.6 процентам

#### Теперь обучим модель с такими же параметрами на всех данных

In [645]:
lr = LogisticRegressionCV(cv = 3)

In [646]:
lr.fit(X, y)

LogisticRegressionCV(cv=3)

### Демонстрация работы на тестовых данных

In [647]:
test = pd.read_csv("test.tsv", sep = "\t")

In [648]:
test

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,0
1,Ночью под Минском на президентской горе Белара...,0
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,0
3,"Сельская церковь, собравшая рекордно низкое ко...",0
4,Акции Google рухнули после объявления о переза...,0
...,...,...
995,Прокуратура заподозрила Явлинского в авторитар...,0
996,В День Победы стратегические ракетоносцы Ту-16...,0
997,СК возбудил дело против авиакомпании «Победа» ...,0
998,Криптомонетный двор Туркменистана выпустил юби...,0


#### Также тестовые данные меняем с помощью Word2vec, получаем valid размерности (1000, 300)

In [649]:
test.title = test.title.apply(preprocess)

In [650]:
test

,title,is_fake
0,роскомнадзор представить реестр сочетание цвет...,0
1,ночью под минский на президентский гора белара...,0
2,бывший спичрайтер юрий лоза рассказать о трудн...,0
3,"сельский церковь, собрать рекордно низкий коли...",0
4,акция google рухнуть после объявление о переза...,0
...,...,...
995,прокуратура заподозрить явлинский в авторитарн...,0
996,в день победа стратегический ракетоносец ту-16...,0
997,ск возбудить дело против авиакомпания «победа»...,0
998,криптомонетный двор туркменистан выпустить юби...,0


In [651]:
texts = test.title.tolist()

In [652]:
line = 0
for line in range(len(texts)):
    for i in punct:
        if i in texts[line]:   
            texts[line] = texts[line].replace(i, "")

In [653]:
vecs = []
errors = []
for line in tqdm(texts):
    vec = 0
    for word in line.split():
        try:
            word_tagged = tag(word)
        except:
            continue
        if word_tagged in word2vec:
            vec += word2vec[word_tagged]
    vecs.append(vec)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1001.80it/s]


In [654]:
len(vecs)

1000

In [655]:
valid = pd.DataFrame(vecs)
valid.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.399243,0.346453,-0.241550,-1.610480,0.930938,0.315715,-0.292521,0.221916,-0.871547,-0.420912,...,-0.050024,-0.173573,0.616107,1.095573,-0.355622,-0.285410,-0.607233,-0.980589,1.691957,0.000119
1,-0.252480,-0.144776,0.015977,-0.933832,0.028789,1.634717,-0.123156,0.780652,0.716374,-0.455380,...,1.280913,-0.328480,-0.421936,-0.488846,0.082575,-0.778517,-0.448440,0.439076,2.287841,0.671776
2,-0.975781,-0.434427,-1.701489,-1.388748,-0.248310,0.791215,-0.399773,0.623801,1.398039,-0.009429,...,1.301376,0.485133,-0.422953,0.946021,-0.660889,1.506385,0.592294,-0.597175,2.133172,-0.379675
3,0.449240,0.149839,0.253881,-2.487186,1.743972,-0.128937,-0.673326,-0.279032,-0.557040,-0.688142,...,2.003700,0.755393,0.446432,1.226905,0.351283,-1.986991,-1.105770,0.686792,0.962600,0.586750
4,1.114974,-0.384749,-0.033885,-0.101726,-0.068298,0.082171,0.152364,-0.478397,-0.147899,0.459366,...,1.091010,-0.312723,-0.597915,-0.199008,0.008621,-0.017089,-1.149785,0.037035,1.027470,-0.314021


#### Строим прогноз на валидационных данных

In [656]:
valid_pred = lr.predict(valid)
test.is_fake = valid_pred

In [657]:
test.to_csv('prediction.tsv', index = False)

In [658]:
prediction = pd.read_csv("prediction.tsv")

#### А вот предикшн на валидационных данных

In [659]:
prediction

,title,is_fake
0,роскомнадзор представить реестр сочетание цвет...,1
1,ночью под минский на президентский гора белара...,1
2,бывший спичрайтер юрий лоза рассказать о трудн...,1
3,"сельский церковь, собрать рекордно низкий коли...",1
4,акция google рухнуть после объявление о переза...,0
...,...,...
995,прокуратура заподозрить явлинский в авторитарн...,1
996,в день победа стратегический ракетоносец ту-16...,1
997,ск возбудить дело против авиакомпания «победа»...,0
998,криптомонетный двор туркменистан выпустить юби...,0
